# Adults Census Model Training with IBM Watson Machine Learning  
## Scikit-learn: Ensemble learning

## Outline

1. Data Preparation
2. Model Training
3. Model Comparison

#### 1. Data Preparation

In [80]:
# Python libraries that we'll use throughout the analysis
import os
import types
import numpy as np
import pandas as pd
from botocore.client import Config
import ibm_boto3

In [81]:
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client = ibm_boto3.client(service_name='s3',
                          ibm_api_key_id='7rULUuPgOFK63b96NYtJ9q1SzM16E4cyWVeUCIg-ukte',
                          ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
                          config=Config(signature_version='oauth'),
endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client.get_object(Bucket='adultscensusproject-donotdelete-pr-7nekr9yeith2lc',Key='adult.csv')['Body']
df = pd.read_csv(body)

df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,IncomeGroup
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [82]:
# summarize the class distribution
import collections

target = df.values[:,-1]
counter = collections.Counter(target)
for k,v in counter.items():
    per = v / len(target) * 100
    print('Class=%s, Count=%d, Percentage=%.3f%%' % (k, v, per))


Class= <=50K, Count=34014, Percentage=75.216%
Class= >50K, Count=11208, Percentage=24.784%


#### 2. Model Training

In [83]:
!pip install watson-machine-learning-client-v4

In [84]:
%reload_ext watermark
%watermark  -d -u -h -m -a "WML Notebook Instance" -v -p numpy,pandas,scipy,sklearn,ibm_boto3,botocore,ibm_watson_machine_learning,watson_machine_learning_client_v4

# The runtime has 1 vCPU and 4 GB RAM.

Author: WML Notebook Instance

Last updated: 2020-12-10

Python implementation: CPython
Python version       : 3.7.9
IPython version      : 7.15.0

numpy                            : 1.18.5
pandas                           : 1.0.5
scipy                            : 1.5.0
sklearn                          : 0.23.1
ibm_boto3                        : 2.7.0
botocore                         : 1.16.11
ibm_watson_machine_learning      : 1.0.43
watson_machine_learning_client_v4: not installed

Compiler    : GCC 7.3.0
OS          : Linux
Release     : 4.15.0-122-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 56
Architecture: 64bit

Hostname: notebook-conda1py3725f880f6ada9447e8629ea063173a606-54c7956bhrd



In [85]:
# Python libraries 
import time
import joblib
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier

In [86]:
start_time = time.time()

#Prepocessing
print('building datasets')
features = 'age workclass fnlwgt education education-num marital-status occupation relationship race sex capital-gain capital-loss hours-per-week native-country'
target = 'IncomeGroup'
X = df[features.split()]
y = df[target]

# label encode the target variable to have the classes 0 and 1
y = LabelEncoder().fit_transform(y)

cat_ix = X.select_dtypes(include=['object', 'bool']).columns
print(f"cat_ix: {cat_ix}")
num_ix = X.select_dtypes(include=['int64', 'float64']).columns
print(f"num_ix: {num_ix}")

#use stratify for un balanced number of examples for each class label
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=42,shuffle=True)
print(f"X_train:{X_train.shape}")
print(f"X_test: {X_test.shape}")
print(f"y_train:{y_train.shape}")
print(f"y_test: {y_test.shape}")

# Model Training
print('training model')
MODEL = 'Ensemble'
    
tree  = DecisionTreeClassifier(max_leaf_nodes=30,
                               criterion="gini", 
                               max_depth=32, 
                               min_samples_leaf=1, 
                               random_state=42)

bag = BaggingClassifier(base_estimator=tree,
                        n_estimators=100,
                        oob_score=True,
                        bootstrap=True,
                        bootstrap_features=False,
                        n_jobs=1,
                        random_state=42)

# define steps
steps = [('c',OneHotEncoder(handle_unknown='ignore'),cat_ix), ('n',MinMaxScaler(),num_ix)]
# one hot encode categorical, normalize numerical
ct = ColumnTransformer(steps)
# wrap the model a pipeline
pipe  = Pipeline(steps=[('transformer',ct),('bag',bag)])
pipe.fit(X_train, y_train)
print(f"model: {pipe}")
  
## Evaluate the model

# using threshold
threshold = 0.5 # set threshold as 0.5
y_pred = (pipe.predict_proba(X_test)[:,1] >= threshold).astype(int) 
print(f"y_pred: {y_pred}")
print(f'Misclassified examples:{(y_test != y_pred).sum()}')
print(pipe.predict_proba(X_test))
  
print(f"distinct y_pred: {np.unique(y_pred)}")
print(f"distinct y_test: {np.unique(y_test)}")
  
# Confusion Matrix
conf_mat = metrics.confusion_matrix(y_test, y_pred)
print(f"confusion matrix:\n {conf_mat}")

# AUC
auc = metrics.roc_auc_score(y_test, y_pred)
print(f"auc: {auc}")

# Accuracy
accuracy = metrics.accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}") 

# Precision
precision = metrics.precision_score(y_test, y_pred)
print(f"Precision: {precision}")

# Recall
recall = metrics.recall_score(y_test, y_pred)
print(f"Recall: {recall}")

# F1
f1 = metrics.f1_score(y_test, y_pred)
print(f"F1: {f1}")

# classification_report
print('classification_report')
print(metrics.classification_report(y_test, y_pred))



time_elapsed = (time.time() - start_time)
print("Computational time:{}".format(time_elapsed))

building datasets
cat_ix: Index(['workclass', 'education', 'marital-status', 'occupation',
       'relationship', 'race', 'sex', 'native-country'],
      dtype='object')
num_ix: Index(['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss',
       'hours-per-week'],
      dtype='object')
X_train:(31655, 14)
X_test: (13567, 14)
y_train:(31655,)
y_test: (13567,)
training model
model: Pipeline(steps=[('transformer',
                 ColumnTransformer(transformers=[('c',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  Index(['workclass', 'education', 'marital-status', 'occupation',
       'relationship', 'race', 'sex', 'native-country'],
      dtype='object')),
                                                 ('n', MinMaxScaler(),
                                                  Index(['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss',
       'hours-per-week'],
      dtype=

#### save model

In [ ]:
"apikey":"xeRVsZz969-qRWj1qpBs_LiSvisLQJ8zoBsXI_pwF0Hw",

In [52]:
wml_credentials = {
                      "apikey":"xeRVsZz969-qRWj1qpBs_LiSvisLQJ8zoBsXI_pwF0Hw",
                      "url": "https://us-south.ml.cloud.ibm.com"
}
from ibm_watson_machine_learning import APIClient
wml_client = APIClient(wml_credentials)

In [53]:
wml_client

In [71]:
space_uid = '437e6cb6-88cf-4168-a40d-d703b8f6c174'

wml_client.set.default_space(space_uid)

'SUCCESS'

Create the model metadata.

In [97]:
import sys
sys.version

'3.7.9 (default, Aug 31 2020, 12:42:55) \n[GCC 7.3.0]'

In [113]:
softwareSpecificationName = "do_12.9"
sofware_spec_uid = wml_client.software_specifications.get_uid_by_name(softwareSpecificationName)
sofware_spec_uid

'75a3a4b0-6aa0-41b3-a618-48b1f56332a6'

In [118]:
# Model Metadata.
meta_props={
    wml_client.repository.ModelMetaNames.NAME: "Adults Census Ensemble model",
    wml_client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: sofware_spec_uid,
    wml_client.repository.ModelMetaNames.TYPE: "scikit-learn_0.22"
}

In [119]:
# Create the model artifact.
model_artifact = wml_client.repository.store_model(model=pipe,
                                                   meta_props=meta_props,
                                                   training_data=X_train,
                                                   training_target=y_train)


Note: Warnings!! :  Model type scikit-learn_0.22 is deprecated. We recommend you use a supported model type. See Supported Frameworks https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/pm_service_supported_frameworks.html.


In [120]:
model_uid = wml_client.repository.get_model_uid(model_artifact)
print("Model UID = " + model_uid)

Model UID = b83517f2-8c94-4c6e-b449-afbf0202cde8


In [121]:
# Details about the model.
model_details = wml_client.repository.get_details(model_uid)
from pprint import pprint
pprint(model_details)

{'entity': {'label_column': 'l1',
            'software_spec': {'id': '75a3a4b0-6aa0-41b3-a618-48b1f56332a6',
                              'name': 'do_12.9'},
            'training_data_references': [{'connection': {'access_key_id': 'not_applicable',
                                                         'endpoint_url': 'not_applicable',
                                                         'secret_access_key': 'not_applicable'},
                                          'id': '1',
                                          'location': {},
                                          'schema': {'fields': [{'name': 'age',
                                                                 'type': 'int64'},
                                                                {'name': 'workclass',
                                                                 'type': 'object'},
                                                                {'name': 'fnlwgt',
                                 

In [122]:
# Display a list of all the models.
wml_client.repository.list_models()

------------------------------------  ----------------------------  ------------------------  -----------------
ID                                    NAME                          CREATED                   TYPE
b83517f2-8c94-4c6e-b449-afbf0202cde8  Adults Census Ensemble model  2020-12-10T03:19:39.002Z  scikit-learn_0.22
05442617-cc5b-4a25-83c2-8a536e043c86  Adults Census Ensemble model  2020-12-10T03:18:29.002Z  scikit-learn_0.22
------------------------------------  ----------------------------  ------------------------  -----------------


In [123]:
# persist model
joblib.dump(pipe, f"{MODEL}.joblib")
client.upload_file(Filename=f"{MODEL}.joblib",
                   Bucket='adultscensusproject-donotdelete-pr-7nekr9yeith2lc',
                   Key=f"{MODEL}.joblib")